### DATA 643 Project 3: Matrix Factorization methods
_Nathan, Angas, Pavan_

_The goal of this assignment is give you practice working with Matrix Factorization techniques._

Your task is implement a matrix factorization method—such as singular value decomposition (SVD) or Alternating Least Squares (ALS) — in the context of a recommender system.

You may approach this assignment in a number of ways. You are welcome to start with an existing recommender system written by yourself or someone else. Remember as always to cite your sources, so that you can be graded on what you added, not what you found.

SVD can be thought of as a pre-processing step for feature engineering. You might easily start with thousands or millions of items, and use SVD to create a much smaller set of "k" items (e.g. 20 or 70).


#### Dataset

We will be using [MovieLens Latest Datasets](https://grouplens.org/datasets/movielens/). GroupLens Research maintains movie rating data sets collected from the website [MovieLens](http://movielens.org). The datasets were collected over various periods of time, depending on the size of the set. For the scope of the project, we will be using _small dataset_ that contains 100,000 ratings and 1,300 tag applications applied to 9,000 movies by 670 users.

#### Matrix Factorization via Singular Value Decomposition

In simple terms _Matrix factorization_ is a process of breaking down a matrix into product of multiple matrices. There are many different ways to factor matrices, for the scope of the project we will be using singular value decomposition and generate recommendations.

$$\begin{equation}
R = U\Sigma V^{T}
\end{equation}$$

where R is users's ratings matrix, $U$ is the user "features" matrix, $\Sigma$ is the diagonal matrix of singular values (essentially weights), and $V^{T}$ is the movie "features" matrix. $U$ and $V^{T}$ are orthogonal, and represent different things. $U$ represents how much users "like" each feature and $V^{T}$ represents how relevant each feature is to each movie.

Matrix $U$ is described as the user 'features' matix because it is composed of the eigenvectors of $R R^T$, which is a square matrix with dimensions equal to the number of users. In that regard it contains information of how similar each user is to each other user. Similarly, $V$ is described as the movie 'features' because it is composed of the eigenvectors of $R^T R$, and will contain information on how each movie related to everyother movie. The diagonal matrix, $\Sigma$ is composed of the related eigenvalues, that parameterize the interaction between the 'user space' and 'movie space', which is why it can be thought of as weights. 

The SVD method allows use to recontruct the R matrix with imcomplete information. This is why in the steps below we must come up with a system to impute missing values, and then normalize the data with the imputed values. Recall in our first project, the TV recommender system we caculated recommendations by calculating a raw average of all the data, and then added in the row and column averages as a form of user and item bias respectively. In the SVD system, Composing R with the normalized data is a more sophisticated method than the raw average, as the eigenvectors contain more accurate data on how users and items relate. The normalization process accounts for the bias, by adjusting each score with respect to an average value, so reversing the normalization process at the end is also key in calculating the estimated score.

To get the lower rank approximation, we take these matrices and keep only the top $k$ features, which we think of as the underlying tastes and preferences vectors.

We will be using _svds_ function from _scipy_ package.

In [1]:
#Load libraries and functions
import numpy as np
import pandas as pd
from scipy.sparse.linalg import svds
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

#Load datasets
ratings = pd.read_csv('https://raw.githubusercontent.com/akulapa/Data643-Week02/master/Data/ratings.csv')
movies = pd.read_csv('https://raw.githubusercontent.com/akulapa/Data643-Week02/master/Data/movies.csv')

In [2]:
#Convert Users as Rows and Movies as Columns 
M_df = ratings.pivot(index = 'userId', columns ='movieId', values = 'rating').fillna(0)

#Convert Movies as Rows and Users as Columns 
U_df = ratings.pivot(index = 'movieId', columns ='userId', values = 'rating').fillna(0)

M_df.head(10)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
#Moves rated by user 1
ratings[(ratings['userId'] == 1)].head(10)

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [4]:
#Convert to matrix
R = M_df.values
R[10:]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [4., 0., 0., ..., 0., 0., 0.],
       [5., 0., 0., ..., 0., 0., 0.]])

In [5]:
#Calculate mean for each user
user_ratings_mean = M_df.mean(axis=1)
user_ratings_mean.head(10)

userId
1     0.005625
2     0.029230
3     0.020075
4     0.097838
5     0.043128
6     0.015828
7     0.033642
8     0.049471
9     0.018641
10    0.018751
dtype: float64

In [6]:
#Calculate mean for each user
user_ratings_mean = np.mean(R, axis = 1)

In [7]:
#Apply mean for each user
#user_bias = user rating - user over all average 
R_bias = R - user_ratings_mean.reshape(-1, 1)

R_bias[10:]

array([[-0.01709685, -0.01709685, -0.01709685, ..., -0.01709685,
        -0.01709685, -0.01709685],
       [-0.01853077, -0.01853077, -0.01853077, ..., -0.01853077,
        -0.01853077, -0.01853077],
       [ 4.97810501, -0.02189499, -0.02189499, ..., -0.02189499,
        -0.02189499, -0.02189499],
       ...,
       [-0.01367748, -0.01367748, -0.01367748, ..., -0.01367748,
        -0.01367748, -0.01367748],
       [ 3.98698434, -0.01301566, -0.01301566, ..., -0.01301566,
        -0.01301566, -0.01301566],
       [ 4.95030885, -0.04969115, -0.04969115, ..., -0.04969115,
        -0.04969115, -0.04969115]])

In [8]:
#Lets start with entire dataset as is
#Get number of rows and columns 
r, c = R.shape

#get min of row or column size, it acts as starting value for k
k = min(r, c)
k = k - 1

#Get SVD values for entire dataset, k value has to smaller value of ratings matrix
U, sigma, Vt = np.linalg.svd(R, full_matrices=False)

In [9]:
#user features
U[10:]

array([[-5.90787704e-03, -7.27312198e-03,  6.85845692e-03, ...,
         2.18383947e-03, -1.17738879e-03,  4.69405895e-05],
       [-5.92008967e-03,  1.98099050e-03,  2.87516708e-04, ...,
         8.29837880e-03, -3.03295033e-03, -9.25766035e-04],
       [-1.59700025e-02, -1.84703385e-02, -9.25063185e-03, ...,
        -4.22564058e-03,  6.51142234e-03,  1.05287787e-03],
       ...,
       [-6.99980178e-03,  1.58302556e-03,  3.86733047e-05, ...,
        -5.58604000e-03, -5.03446825e-03,  2.39356104e-03],
       [-1.27201885e-02, -1.12626170e-03, -1.50645996e-02, ...,
         1.49270024e-02, -2.55571016e-03,  6.44863095e-03],
       [-4.02662085e-02, -1.91584071e-02, -3.24647253e-03, ...,
        -1.52620874e-03, -5.50592866e-04,  3.23609829e-03]])

In [10]:
#diagonal matrix of singular values
sigma[:10]

array([517.58313979, 243.76943485, 204.30617832, 162.47028777,
       156.30956977, 145.23455301, 136.81751883, 122.99256885,
       118.74152381, 116.32873458])

In [11]:
#movie features
Vt[:10]

array([[-7.89608026e-02, -3.21870213e-02, -1.28461428e-02, ...,
        -6.10278612e-05, -3.66167167e-05, -1.64855666e-03],
       [-3.52425907e-02, -1.52635806e-02,  5.63629358e-03, ...,
        -2.54528542e-04, -1.52717125e-04,  3.94405929e-03],
       [-5.09072327e-02, -6.43081958e-02, -2.84398301e-02, ...,
        -6.24198136e-05, -3.74518881e-05,  9.31325318e-03],
       ...,
       [ 6.36119034e-02,  3.81599907e-02,  6.11142680e-03, ...,
         9.45745474e-05,  5.67447284e-05, -2.23955483e-04],
       [-2.52741783e-02,  7.07321948e-03,  7.16503962e-03, ...,
        -8.13433325e-04, -4.88059995e-04, -5.75903025e-03],
       [ 1.88130162e-02,  8.54217528e-03,  3.80146066e-03, ...,
        -1.18215948e-04, -7.09295690e-05, -2.38136288e-02]])

In [12]:
#Get diagonal sigma
sigma_diag = np.diag(sigma)

#Recalculate ratings
predicted_ratings = np.dot(np.dot(U, sigma_diag), Vt) + user_ratings_mean.reshape(-1, 1)

#Convert to dataframe
predicted_ratings_T = pd.DataFrame(predicted_ratings.T)
predicted_ratings_T['movieId'] = U_df.index

#Movies rated by user 1
userMoviesId = list(ratings[(ratings['userId'] == 1)]['movieId'])

#Predicted values for user 1 
predicted_ratings_T = predicted_ratings_T[predicted_ratings_T.movieId.isin(userMoviesId)]
predicted_ratings_T = predicted_ratings_T[[0,'movieId']].head(10)
predicted_ratings_T

,0,movieId
30,2.505625,31
833,3.005625,1029
859,3.005625,1061
906,2.005625,1129
931,4.005625,1172
1017,2.005625,1263
1041,2.005625,1287
1047,2.005625,1293
1083,3.505625,1339
1087,2.005625,1343


Above are predicted movie ratings for _user-1_. Ratings are very close to actual ratings. Root mean square error is very low, $0.10$. Low error is expected as we used _k_ value as full set. It also suggests that is is _overfitting_. As we lower _k_ value error is expected to increase.

In [13]:
rmse = mean_squared_error(predicted_ratings, R)**0.5
print("RMSE : " + str(rmse))

RMSE : 0.10254517826093584


In [14]:
#Lets loop through reducing k value
k = min(r, c)
for i in range(5, 100, 5):

    # take columns less than k from U
    U_p = U[:,:k]
    # take rows less than k from V
    V_p = Vt[:k,:]
    # build the new S matrix with top k diagnal elements
    S_p = np.zeros((k, k), int)
    for j in range(k):
        S_p[j][j] = sigma[j]
    
    #Recalculate ratings
    predicted_ratings = np.dot(np.dot(U_p, S_p), V_p) + user_ratings_mean.reshape(-1, 1)

    #Calculate error difference
    diffM = R - predicted_ratings
    
    #Frobenius Norm
    frobeniusNorm = np.linalg.norm(diffM, 'fro')
    
    #Singular value ratio has to be 90%
    if (k == min(r, c)):
        sigma_ratio = round(sum(sigma**2)/sum(sigma**2),3)
    else:
        less_singular_values = sigma[ np.where( sigma >= i ) ]
        sigma_ratio = round(sum(less_singular_values**2)/sum(sigma**2),3)
    
    
    #RMSE
    rmse = mean_squared_error(predicted_ratings, R)**0.5
    print("RMSE : " + str(round(rmse,3)) + 
          ' Frobenius Norm : ' + str(round(frobeniusNorm,3)) + 
          ' k-Value reduced by : ' + str(min(r, c) - k) + 
          ' Singlar Value Ratio : ' + str(sigma_ratio)
         )
    
    
    
    #Eliminate rows with low sigma value
    k = min(r, c) - sigma[ np.where( sigma < i ) ].size

RMSE : 0.103 Frobenius Norm : 253.035 k-Value reduced by : 0 Singlar Value Ratio : 1.0
RMSE : 0.103 Frobenius Norm : 253.287 k-Value reduced by : 8 Singlar Value Ratio : 0.997
RMSE : 0.106 Frobenius Norm : 260.598 k-Value reduced by : 66 Singlar Value Ratio : 0.986
RMSE : 0.117 Frobenius Norm : 288.201 k-Value reduced by : 161 Singlar Value Ratio : 0.965
RMSE : 0.136 Frobenius Norm : 334.329 k-Value reduced by : 255 Singlar Value Ratio : 0.938
RMSE : 0.157 Frobenius Norm : 386.067 k-Value reduced by : 329 Singlar Value Ratio : 0.901
RMSE : 0.181 Frobenius Norm : 445.878 k-Value reduced by : 395 Singlar Value Ratio : 0.861
RMSE : 0.205 Frobenius Norm : 504.535 k-Value reduced by : 448 Singlar Value Ratio : 0.816
RMSE : 0.228 Frobenius Norm : 561.137 k-Value reduced by : 491 Singlar Value Ratio : 0.769
RMSE : 0.25 Frobenius Norm : 616.41 k-Value reduced by : 527 Singlar Value Ratio : 0.718
RMSE : 0.272 Frobenius Norm : 670.873 k-Value reduced by : 558 Singlar Value Ratio : 0.674
RMSE : 0

_RMSE_, increases as we reduce of _k-value_. Based on the video https://www.youtube.com/watch?v=c7e-D2tmRE0&index=49&list=PLLssT5z_DsK9JDLcT8T62VtzwyW9LNepV, single value ratio holds good even when we reduce 329 dimensions. 

Let's test _user-1_ prediction once again.

In [15]:
#Number of dimensions
k = 342
# take columns less than k from U
U_p = U[:,:k]
# take rows less than k from V
V_p = Vt[:k,:]
# build the new S matrix with top k diagnal elements
S_p = np.zeros((k, k), int)
for j in range(k):
    S_p[j][j] = sigma[j]

#Recalculate ratings
predicted_ratings = np.dot(np.dot(U_p, S_p), V_p) + user_ratings_mean.reshape(-1, 1)

#Convert to dataframe
predicted_ratings_T = pd.DataFrame(predicted_ratings.T)
predicted_ratings_T['movieId'] = U_df.index

#Movies rated by user 1
userMoviesId = list(ratings[(ratings['userId'] == 1)]['movieId'])

#Predicted values for user 1 
predicted_ratings_T = predicted_ratings_T[predicted_ratings_T.movieId.isin(userMoviesId)]
predicted_ratings_T = predicted_ratings_T[[0,'movieId']]
predicted_ratings_T = pd.merge(predicted_ratings_T, movies, on='movieId', how='inner')
predicted_ratings_T = predicted_ratings_T[[0, 'movieId', 'title', 'genres']]

predicted_ratings_T.columns = ['Rated','movieId', 'Watched','genres']
predicted_ratings_T.sort_values('Rated', ascending = False).head(20)

,Rated,movieId,Watched,genres
4,1.311221,1172,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
8,1.047481,1339,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
13,0.792882,2105,Tron (1982),Action|Adventure|Sci-Fi
19,0.744160,3671,Blazing Saddles (1974),Comedy|Western
7,0.729832,1293,Gandhi (1982),Drama
10,0.725054,1371,Star Trek: The Motion Picture (1979),Adventure|Sci-Fi
12,0.723276,1953,"French Connection, The (1971)",Action|Crime|Thriller
16,0.693549,2294,Antz (1998),Adventure|Animation|Children|Comedy|Fantasy
0,0.646934,31,Dangerous Minds (1995),Drama
6,0.616048,1287,Ben-Hur (1959),Action|Adventure|Drama


In [16]:
ratings = pd.merge(ratings, movies, on='movieId', how='inner')
ratings =ratings.drop(['timestamp'],axis=1)
ratings[(ratings['userId'] == 1)].sort_values('rating', ascending = False).head(20)

,userId,movieId,rating,title,genres
165,1,1172,4.0,Cinema Paradiso (Nuovo cinema Paradiso) (1989),Drama
581,1,2105,4.0,Tron (1982),Action|Adventure|Sci-Fi
535,1,1953,4.0,"French Connection, The (1971)",Action|Crime|Thriller
351,1,1339,3.5,Dracula (Bram Stoker's Dracula) (1992),Fantasy|Horror|Romance|Thriller
849,1,3671,3.0,Blazing Saddles (1974),Comedy|Western
42,1,1029,3.0,Dumbo (1941),Animation|Children|Drama|Musical
84,1,1061,3.0,Sleepers (1996),Thriller
628,1,2150,3.0,"Gods Must Be Crazy, The (1980)",Adventure|Comedy
759,1,2455,2.5,"Fly, The (1986)",Drama|Horror|Sci-Fi|Thriller
0,1,31,2.5,Dangerous Minds (1995),Drama


We can see in the dataframes above, the top one being watched recommedations in our system, the bottom being user 1's actual ratings.  Both sorted by greatest to least return the same titles in a similar order. 	Cinema Paradiso (Nuovo cinema Paradiso) (1989) is at the top of both lists, Bram Stoker's Dracula (1992), Tron (1982), and Blazing Saddles (1974) are also high on the list. The system rates Escape from New York (1981) and Beavis and Butt-Head Do America (1996) low which is agreement with the actual ratings,  but rates Antz (1998) and Time Bandits (1981), both rated 2.0 by the user, but appearing in the recommendation system in the middle of the list.

Since this is small dataset, predicted rating values have changed a lot. Following are the recommendation for _user-1_.

In [17]:
#Convert to dataframe
predicted_ratings_T = pd.DataFrame(predicted_ratings.T)
predicted_ratings_T['movieId'] = U_df.index

userMovies = list(ratings[(ratings['userId'] == 1)]['movieId'])

predicted_ratings_T = predicted_ratings_T[~predicted_ratings_T.movieId.isin(userMovies)]
predicted_ratings_T = predicted_ratings_T[[0,'movieId']]

#Predicted values for user 1 
predicted_ratings_T = pd.merge(predicted_ratings_T, movies, on='movieId', how='inner')
predicted_ratings_T = predicted_ratings_T[[0, 'movieId', 'title', 'genres']]
predicted_ratings_T.columns = ['Predictions','movieId', 'Recommendation','genres']

predicted_ratings_T = predicted_ratings_T.sort_values('Predictions', ascending = False)
predicted_ratings_T.head(10)

,Predictions,movieId,Recommendation,genres
1104,0.445184,1375,Star Trek III: The Search for Spock (1984),Action|Adventure|Sci-Fi
1031,0.382872,1283,High Noon (1952),Drama|Western
1728,0.367842,2194,"Untouchables, The (1987)",Action|Crime|Drama
1024,0.366594,1276,Cool Hand Luke (1967),Drama
1785,0.357895,2278,Ronin (1998),Action|Crime|Thriller
959,0.356547,1208,Apocalypse Now (1979),Action|Drama|War
2690,0.353055,3396,"Muppet Movie, The (1979)",Adventure|Children|Comedy|Musical
2984,0.349767,3755,"Perfect Storm, The (2000)",Drama|Thriller
929,0.339345,1175,Delicatessen (1991),Comedy|Drama|Romance
2417,0.338687,3033,Spaceballs (1987),Comedy|Sci-Fi


Looking at the top list of unwatched movies above, there do seem to be some connections. The user had a luke-warm rating for Star Trek the Motion Picture of 2.5, and a Star Trek movie (Search for Spock) tops the list. The consensus for Star Trek fans is that the for The Original Series movies, the even numbered ones (II: Wrath of Khan{71}, IV: Voyage Home{67}, VI: Undiscovered Country{65}) are better than the odd numbered ones (I: The Motion Picture{48}, III: Search for Spock{55}, V: Final Frontier{43}). The meta-critic scores are in the brakets. So, it did pick a better Star Trek movie by general consensus, just not a good one. 

Spaceballs (1987) is recommended, and the user rated Blazing Saddles favorably (3.0). Both are Mel Brooks comedies. High Noon (1952), Untouchables, The (1987), Cool Hand Luke (1967), Ronin (1998), and Apocalypse Now (1979) all seem to make sense (at least to Nathan) with respect to how highly the user rated French Connection, The (1971), and Sleepers (1996), although (Nathan finds) it is hard to articulate why.  

#### Conclusion
- Singular value decomposition, generates recommendation accurately.
- SVD does not work well when we have missing values.

#### References
- http://web.mit.edu/be.400/www/SVD/Singular_Value_Decomposition.htm
- https://beckernick.github.io/matrix-factorization-recommender/
- https://github.com/beckernick/matrix_factorization_recommenders
- https://www.youtube.com/watch?v=yLdOS6xyM_Q&feature=youtu.be&list=PLLssT5z_DsK9JDLcT8T62VtzwyW9LNepV
- https://www.youtube.com/watch?v=c7e-D2tmRE0&index=49&list=PLLssT5z_DsK9JDLcT8T62VtzwyW9LNepV
- https://hackernoon.com/introduction-to-recommender-system-part-1-collaborative-filtering-singular-value-decomposition-44c9659c5e75
- https://github.com/mutaphore/svd-image-compression/blob/master/image_compression.py
- http://nicolas-hug.com/blog/matrix_facto_3
- http://metacritic.com/pictures/star-trek-movies-ranked-worst-to-best/1